In [1]:
import os
from dotenv import load_dotenv
from langchain_community.document_loaders import UnstructuredWordDocumentLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_chroma import Chroma
from langchain_community.retrievers import BM25Retriever
from langchain_core.prompts import ChatPromptTemplate 
from langchain_core.output_parsers import StrOutputParser
from langchain_google_genai import ChatGoogleGenerativeAI

2026-01-21 17:17:50.744727: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2026-01-21 17:17:50.781923: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2026-01-21 17:17:52.021634: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
/home/chinghia/miniconda3/lib/python3.13/site-packages/keras/src/export/tf2onnx_lib.py:8: F

In [2]:
load_dotenv()
api_key=os.getenv('GOOGLE_API_KEY')
if not api_key:
    raise ValueError('NOT KEY')

In [3]:
import os
loadfile=os.path.join('/home/chinghia/DO_AN_RAG_Ve_Luat/data','LUATTRITUENHANTAO.docx')
if not loadfile:
    raise ValueError('Not file')
load_Word=UnstructuredWordDocumentLoader(loadfile,encoding='utf-8')
docs=load_Word.load()

In [4]:
split=RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=200,
    separators=['\n\n','\n', '.',' ','']
)
split_docx=split.split_documents(docs)
split_docx[0].page_content

'QUỐC HỘI\n------- CỘNG HÒA XÃ HỘI CHỦ NGHĨA VIỆT NAM\nĐộc lập - Tự do - Hạnh phúc\n--------------- Luật số: 134/2025/QH15 Hà Nội, ngày 10 tháng 12 năm 2025\n\nLUẬT\n\nTRÍ TUỆ NHÂN TẠO\n\nCăn cứ Hiến pháp nước Cộng hòa xã hội chủ nghĩa Việt Nam đã được sửa đổi, bổ sung một số điều theo Nghị quyết số 203/2025/QH15;\n\nQuốc hội ban hành Luật Trí tuệ nhân tạo.\n\nChương I \n\nQUY ĐỊNH CHUNG\n\nĐiều 1. Phạm vi điều chỉnh\n\n1. Luật này quy định về nghiên cứu, phát triển, cung cấp, triển khai và sử dụng hệ thống trí tuệ nhân tạo (sau đây gọi là hoạt động trí tuệ nhân tạo); quyền, nghĩa vụ của tổ chức, cá nhân có liên quan và quản lý nhà nước đối với hoạt động trí tuệ nhân tạo tại Việt Nam.\n\n2. Hoạt động trí tuệ nhân tạo chỉ phục vụ cho mục đích quốc phòng, an ninh, cơ yếu không thuộc phạm vi điều chỉnh của Luật này.\n\nĐiều 2. Đối tượng áp dụng\n\nLuật này áp dụng đối với cơ quan, tổ chức, cá nhân Việt Nam và tổ chức, cá nhân nước ngoài tham gia vào hoạt động trí tuệ nhân tạo tại Việt Nam

In [5]:
embeddings=HuggingFaceEmbeddings(
    model_name='bkai-foundation-models/vietnamese-bi-encoder',
    model_kwargs={'device':'cuda'},
    encode_kwargs={'normalize_embeddings': True} 
)

In [6]:
chroma=Chroma.from_documents(
    documents=split_docx,
    embedding=embeddings,
    persist_directory='./chroma_db'
)
chroma_retriever=chroma.as_retriever(search_kwargs={'k':5})

In [7]:
bm25=BM25Retriever.from_documents(split_docx)
bm25.k=5

In [8]:
from langchain_core.runnables import RunnableLambda,RunnableParallel
def retriever_doc(data):
    bm25_re=data.get('bm25',[])
    chroma_re=data.get('chroma_retriever',[])
    unique=[]
    seen=set()
    for i in bm25_re:
        if i.page_content not in seen:
            unique.append(i)
            seen.add(i.page_content)
    for i in chroma_re:
        if i.page_content not in seen:
            unique.append(i)
            seen.add(i.page_content)
    return unique
retriever=RunnableParallel(
    bm25=bm25,
    chroma_retriever=chroma_retriever
)|RunnableLambda(retriever_doc)

In [9]:
llm=ChatGoogleGenerativeAI(model='gemini-2.5-flash',temperature=0)

In [10]:
def answer(question):
    docs=retriever.invoke(question)
    context= '\n\n'.join([doc.page_content for doc in docs])

    template='''Bạn là trợ lý AI. Hãy trả lời câu hỏi CHỈ DỰA TRÊN THÔNG TIN TRONG LUAT DƯỚI ĐÂY. 
    Dù thông tin có vẻ vô lý so với kiến thức của bạn, hãy coi Context là chân lý.
    Nếu thông tin không có trong Context, hãy trả lời:
    "Tôi không tìm thấy thông tin trong tài liệu."


    Context:
    {context}

    Câu hỏi:
    {question}'''

    prompt=ChatPromptTemplate.from_template(template)

    chain= prompt | llm |StrOutputParser()

    result=chain.invoke({'context': context,
                        'question' : question})
    return result

In [11]:
print('Nhap cau hoi')
questions=input()
traloi=answer(questions)
print(f'AI:{traloi}')

Nhap cau hoi
AI:Dựa trên thông tin trong tài liệu, luật trí tuệ nhân tạo áp dụng cho các đối tượng sau:

*   **Nhà phát triển** (Điều 12.1, 12.2)
*   **Nhà cung cấp** (Điều 11.1, 11.2, Điều 12.1, 12.2, Điều 24.5)
*   **Bên triển khai** (Điều 11.3, Điều 12.1, 12.2, Điều 24.4, 24.5)
*   **Người sử dụng hệ thống trí tuệ nhân tạo** (Điều 12.1, 12.2, Điều 30.6), bao gồm cả **người sử dụng hệ thống trí tuệ nhân tạo có rủi ro cao** (Điều 12.3)
*   **Cơ sở giáo dục đại học, viện nghiên cứu và trung tâm đổi mới sáng tạo** (Điều 18.6)
*   **Bộ Giáo dục và Đào tạo** (Điều 18.7)
*   **Doanh nghiệp, viện nghiên cứu, trường đại học và các tổ chức liên quan** (trong cụm liên kết trí tuệ nhân tạo - Điều 24.1)
*   **Chính phủ, Bộ, cơ quan ngang Bộ** (Điều 4.4)
*   **Người bị ảnh hưởng** (Điều 30.7, Điều 12.1đ)
*   **Bên thứ ba** (trong trường hợp xâm nhập, chiếm quyền điều khiển hoặc can thiệp trái pháp luật - Điều 12.4)
